# Projekt 6 - Videó streaming

Manapság már nagyon sok olyan honlap létezik, ahol élőben nézhetünk kamera képeket valamiről, pl. valaki közvetíti ahogy alszik, Abbey road-i átkelőhely élő képe (ahol a Beatles sétált), vagy akár ürrakéták kilövése. Egy sokkal hétköznapiasabb dologra is lehet használni, lakásunk/házunk megfigyelése távolból. Léteznek térfigyelő kamerák, amivel az ingatlanunk biztonságát erősíthetjük, de akár mi is megcsinálhatjuk ezt egy egyszerű webcamerával. A Raspberry Pi-on keresztül élőben közvetíthetjük a megfigyelt terület képét egy IP címre, amit aztán bármilyen internetet elérő eszközről, pl. mobil telefon, megnézhetünk. 

## Mit fogsz készíteni?

Egy webkamerából (vagy Picam-ből) és egy Raspberry Pi-ból álló rendszert rakunk össze, ahol az utóbbi egy szerverként fog szolgálni. A megadott IP címet meglátogatva láthatjuk majd a kamera képét. 

## Mit tanulsz meg?

A streamingelős projekt elkészítésével a következőket tanulod meg:

* Hogyan használd az ```opencv``` csomagot a webkamerával való kommunikálásra.
* Hogyan használjuk a ```flask``` csomagot streamingre.
* Mire való a ```yield``` parancs.
* Mik azok a *dekorátorok*.
* Megismerkedünk néhány alapvető *HTML* paranccsal.

## A projekt részletekre bontása

* Elkészíteni az áramkört.
* Beimportálni a csomagokat amik segítik a munkánkat: ```cv2```, ```flask```.
* Definiálni egy ```flask``` applikációt.
* Definiálni az *index* oldalt a streaming honlaphoz.
* Megírni az *index*hez tartozó *HTML* kódot. 
* Definiálni függvényeket a kamera kép megjelenítésére.

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek)  (vagy laptop)

b) [Webkamera](https://www.emag.hu/iuni-k6i-webkamera-full-hd-1080p-mikrofonnal-usb-2-0-plug-play-515422/pd/DX66N2MBM/?cmpid=87141&gclid=CjwKCAjwj6SEBhAOEiwAvFRuKL7E3Z6v7Ei_MNy1eFxoAn4ySFojVRVyiqf8BByR43dhONUlKDsrPBoC4sIQAvD_BwE) vagy [Picam](https://malnapc.hu/raspberry-pi-camera-board-v2-8mp)

## A kapcsolási rajz

A kameránkat kössük össze a Raspberry Pi-jal.

## A kód

Nyissunk meg egy új python fájlt és mentsük el pl. ```streaming.py``` név alatt. 

Ebben a projektben mi magunk fogjuk megírni a szerver oldali kódot a kamera képének megjelenítésére és ehhez a *Flask* nevű csomagot fogjuk használni. A *Flask* az egy mikor webkészítő keret Pythonban írva.

Első lépésként importáljuk be a megfelelő csomagokat amikre szükségünk lesz.

### Importálások

Első lépésként beimportáljuk a szükséges csomagokat:

* ```cv2``` - webkamerával való kommunikálásra az opencv csomag.

A ```flask``` csomagból pedig beimportáljuk a következőket:

* ```Flask``` - segít a készítendő applikáció objektumának létrehozásában. 
* ```render_template``` - ez a függvény hozza létre a megjelenítendő HTML kódot az argumentumba megadott html fájlunk alapján.
* ```Response``` - ez az objektum segít abban, ha a html-ből jövő adattal valamit kell kezdeni.

A fő programunkban, így a beimportálások a következőképp néznek ki:

```streaming.py```:

In [2]:
from flask import Flask, render_template, Response
import cv2

### Az első honlapunk

A *Flask* csomag a honlapkészítésnél különválasztja a háttérben végzendő műveleteket és magát a weblapnak a megjelenítését. Ez annyiból áll, hogy írunk egy Python kódot ami elindít egy szervert a weblap megjeleníttetésére, illetve külön lekódoljuk a weblap html alapját, amit a szerver majd megnyit kérés esetén. 

A *Flask*ban az első dolog, hogy a ```Flask``` objektummal létrehozunk egy applikációs objektumot, ```app = Flask(__name__)```, amivel többek között majd elindíttatjuka  szervert. Az argumentumban szereplő ```__name___``` az egy globális változó, ami tárolja a jelenlegi modulunk nevét, amennyiben az interpreterben probáljuk megnézni az értékét, akkor ```___main___``` értéket kapunk vissza.

In [1]:
__name__

'__main__'

Weblapokat függvényeken keresztül tudunk megjeleníttetni, és lényegében egy oldalhoz egy függvényt szoktak rendelni. A következő lépésben létrehozzuk az ```index()``` függvényt, aminek a feladata az lesz, hogy megjelenítse az ```index.html``` fájlt weblap formájában. Erre a ```render_template('index.html')``` függvényt használjuk, ahol az argumentumban a megjelenítendő fájl nevét írjuk be. 

A függvény elé még meg kell adnunk, hogy az alap weblapchez képest, melyik URL-en próbáljuk majd megjelenítettni a weblapot. Ezt a függvény elé írt dekorátorral tudjuk megtenni, ```@app.route('/')```, ahol a ```@``` jelzi, hogy egy dekorátort használunk, aminek a neve ```app.route```, azaz ami létrehozza az URL-t és argumentumként megadjuk, hogy a fő URL címet akarjuk megjeleníteni, ```'/'```. 

Már csak el kell indítanunk a szervert a fő részben, ami a saját számítógépünk lesz. Az ```app.run(host='0.0.0.0',port='5000', debug=True)``` parancs indítja a szervert. A ```host``` paraméternél megadhatjuk, hogy milyen IP címen jelenjen meg, a ```0.0.0.0``` cím a *localhost*ra utal, azaz a saját számítógépünk lenne a szerver, ```port``` paraméterrel pedig, hogy melyik porton. 

```streaming.py```:

In [ ]:
from flask import Flask, render_template, Response
import cv2

app = Flask(__name__)

@app.route('/')
def index():
    # rendering webpage
    return render_template('index.html')

if __name__ == '__main__':
    # defining server ip address and port
    app.run(host='0.0.0.0',port='5000', debug=True)

Ezután csak egy html oldalt kell elkészítenünk amit a ```templates``` mappába mentünk el ```index.html``` név alatt. A html kódolás szabályait itt nem részleteznénk. A lenti kód egy címet ad a weblapnak, amit a böngészőnk fülecskéjén lehet majd olvasni, illetve magán a weblapon megjelenik a híres ```Hello, World!``` szöveg.

```templates/index.html```:

In [ ]:
<html>
  <head>
    <title>Video Streaming Demonstration</title>
  </head>
  <body>
    <h1>Hello, World!</h1>
  </body>
</html>

Hogy leteszteljük a programunkat, le kell futtatnunk a ```python streaming.py``` parancsot, ami elindítja a lokális szerverünket a számítógépen. Ezután egy böngészőbe beírva a ```localhost:5000``` címet, meg is jelenik az egzszerű weblapunk. A szerverünket a paranccsorban tudjuk megállítani, ha lenyomjuk a ```CTRL+C``` billentyűkombinációt (akár többször is). 

### Mi a *yield*

### Streamingelés

Ahhoy, hogy a kamera képét jelenítsük meg, egy szöveg helyett, lényegében nem kell sokat változtatnunk. Azt a megközelítést alkalmazzuk, hogy folytonosan rákérdezünk a szerverre, hogy küldje el nekünk az aktuális képet amit lát. Így a szerver csak akkor készít képet, amikor megkérik rá, nem terheli le magát fölösleges adatáramlatokkal. Emellett a már megjelenített kép mellé nem készít újat, hanem felülírja azt, azaz nagyon gyorsan jelenít meg álló képeket ugyanazon a helyen így keltve mozizós hatást. Htmlben ezt úgy érjük el, hogy a fejlécben (header fájl) a ```Content-Type``` paramétert ```multipart/x-mixed-replace```-re definiáljuk és a ```boundary``` paramétert pedig ```frame```-é tesszük egyenlővé. 

*Flask*ban a legegyszerűbb a *Motion JPEG* módszert használni a fenti elméletre. Ez képes gyorsan megjeleníteni az egymás utáni képeket, viszont relatív rossz lesz a kép minősége, hiszen az elkészített képünket át kell alakítanunk JPEG formátumba. Nézzük, hogy néz ki gyakorlatban a kód.

```streaming.py```:

In [ ]:
from flask import Flask, render_template, Response
import cv2

app = Flask(__name__)
#print(cap.isOpened())

@app.route('/')
def index():
    # rendering webpage
    return render_template('index_test.html')

def gen(camera):
    while True:
        #get camera frame
        ret, frame = camera.read()
        ret, frame = cv2.imencode('.jpg', frame)
        frame = frame.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')

@app.route('/video_feed')
def video_feed():
    cap = cv2.VideoCapture('/dev/video0')   # cap = cv2.VideoCapture(0)
    cap.open('/dev/video0')                 # cap.open(0)
    return Response(gen(cap),
                    mimetype='multipart/x-mixed-replace; boundary=frame')


if __name__ == '__main__':
    # defining server ip address and port
    app.run(host='0.0.0.0',port='5000', debug=True)

Ehhez tartozik a ```templates/index.html``` fájl, amihez új tartalmat adtunk. Az újdonság az ```<img id="bg" src="{{ url_for('video_feed') }}">``` sor ami a kép megjelenítéséért felel. A ```src``` paraméter adja meg, hogy honnan jelenítse meg a weblap a képet. Ennek értéke egy függvény, ami legenerálja nekünk az URL-t, ```{{ url_for('video_feed') }}```. A ```video_feed``` az a ```streaming.py``` programban egy függvény, ami a kép elkészítéséért és megfelelő formátumba alakításáért felel, hogy html kompatibilis legyen. 

```templates/index.html```

In [ ]:
<html>
  <head>
    <title>Video Streaming Demonstration</title>
  </head>
  <body>
    <h1>Video Streaming Demonstration</h1>
    <img id="bg" src="{{ url_for('video_feed') }}">
  </body>
</html>

A programunkat két függvénnyel bővítjük ki, a ```gen(camera)``` és a ```video_feed()``` fügvényekkel. Nézzük melyik mit csinál. Miután az ```index()``` függvény legenerálja nekünk a weboldalunkat, az ```img``` tag lekéri a ```video_feed``` függvényhez tartozó URL-t és vele együtt az elkészített képet is. Az URL-t a ```@app.route('/video_feed')``` dekorátor rendeli a fügvényhez. 

A függvényen belül a már megszokott módon megnyitjuk a kommunikációt a kameránkkal. Linux alatt úgy találtam, hogy a ```cap = cv2.VideoCapture('/dev/video0')``` működik, míg Windows alatt a ```cap = cv2.VideoCapture(0)```. Majd a függvény vissza ad egy ```Response``` objektumot, választ a honlap kép kérésére, ahol paraméterként megadja a képet, amit a ```gen(cam)``` függvény generál le, illetve a ```mimetype``` paraméternek megmondja, hogy a képet az előző kép helyére szeretné tenni egy *frame*-be, ```Response(gen(cap), mimetype='multipart/x-mixed-replace; boundary=frame')```.

A ```gen``` függvény bemenő paramétere a kamera objektuma. A függvényen belül, egy végtelen ciklus elindítása után, készít egy képet, ```ret, frame = camera.read()```, azt átalakítja jpeg formátumnak megfelelővé, ```ret, frame = cv2.imencode('.jpg', frame)```, majd azt byte-okba, ```frame = frame.tobytes()```. Az így kapott byte sort beilleszti egy szabványosított képmegjelenítési byte formába, ```yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')``` és ezt adja vissza válaszul a ```gen``` függvény. 

Amíg a *Flask* ```debug``` üzemmódban van, addig lényegében csak egy böngészőt képes kiszolgálni, azaz csak egy valaki képes nézni a valós streaming képet. Ezt többféleképp is lehet módosítani, de ez már nem ennek a projektnek a témaköre.

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```streaming.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```streaming.py```-t elmentettük. Ott begépelve a ```python streaming.py``` parancsot, letesztelhetjük a programunk működését. Ha minden jól megy akkor a program elindítása után, elindul egy szerver, majd egy böngészőt megnyitva és a begépelve a ```localhost:5000``` URL-t, az index oldal tartalma kell megjelenjen a webkamera képével. 

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* Az *opencv* könyvtárat felhasználva módosítsuk a kamera képét (pl. szürke kép, filterelés stb.) és azt jelenítsük meg a honlapon.
* Adaptáljuk a streamingelős kódunkat úgy, hogy egyben mozgásérzékelő is legyen. Ha a képen mozgás történik, akkor a honlapon jelenjen meg egy felirat erről.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_tutorials.html

2) flask - https://flask.palletsprojects.com/en/2.0.x/

3) yield - https://realpython.com/introduction-to-python-generators/

4) decorator - https://realpython.com/primer-on-python-decorators/

5) flask - https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-i-hello-world

6) streaming - https://blog.miguelgrinberg.com/post/video-streaming-with-flask